# House Prices: Advanced Regression Techniques

In [1]:
import pandas as pd
import numpy as np

In [34]:
all_train_set = pd.read_csv('data/train.csv')
all_test_set = pd.read_csv('data/test.csv')

In [17]:
all_train_set.iloc[[1,2,3]]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000


In [36]:
train_set = all_train_set.fillna({
    'LotFrontage': 0.0
})

In [6]:
test_set.shape

(1459, 80)

## Validate a Model

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [40]:
model = LinearRegression()
kfold = KFold(5)
np.random.seed(0)

for (train_idx, cv_idx) in kfold.split(train_set):
    construct = train_set.iloc[train_idx]
    validate = train_set.iloc[cv_idx]
    
    construct_X = construct[['LotFrontage', 'LotArea']]
    construct_y = construct['SalePrice']
    
    validate_X = validate[['LotFrontage', 'LotArea']]
    validate_y = validate['SalePrice']
    
    model.fit(construct_X, y=construct_y)
    predictions = model.predict(validate_X)
    rmse = np.sqrt(mean_squared_error(validate_y, predictions))
    print(rmse)

69166.01860003558
82187.87790288772
81604.42393262105
69205.29999784389
78931.17922971677


In [ ]:
71300.71521536467
84891.20609629959
83025.22668772546
69539.42683754288
79158.31206520787